<a href="https://colab.research.google.com/github/ByRocking/JocGrafica/blob/main/Copy_of_mib_lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laborator 6

În cadrul acestui laborator, vom continua studiul algoritmilor evolutivi, concentrându-ne în detaliu pe diferite moduri de reprezentare a soluțiilor și pe algoritmi specializați.

Vom începe prin aprofundarea reprezentării de tip permutație, soluționând [problema damelor](https://ro.wikipedia.org/wiki/Problema_damelor) (varianta generală) cu biblioteca DEAP și analizând operatorii specifici.

Apoi, vom introduce o tehnică puternică și flexibilă: reprezentarea cu [chei aleatoare (random keys)](https://kalami.medium.com/understanding-random-key-encoding-a-simple-approach-to-solving-complex-optimization-problems-ab17ee028f66). Vom explora cum funcționează aceasta și vom testa metoda [Biased Random Key Genetic Algorithm (BRKGA)](https://www.sciencedirect.com/science/article/pii/S0377221724002303) folosind biblioteca [`pymoo`](https://pymoo.org/algorithms/soo/brkga.html) pentru aceeași problemă a damelor.

## Partea I: Reprezentarea prin permutări cu DEAP

Încă odată, începem prin instalarea DEAP, folosind comanda `pip install`:

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.8 MB/s eta 0:00:00


Importarea componentelor și librăriilor necesare:

In [ ]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import numpy as np
import random

### Problema celor N dame

Conform [descrierii](https://ro.wikipedia.org/wiki/Problema_damelor):
Problema damelor implică plasarea a N dame pe o tablă de șah de NxN astfel încât să nu existe două dame care se amenință reciproc. Aceasta înseamnă că nicio pereche de două dame nu trebuie să fie pe aceeași linie, pe aceeași coloană sau pe aceeași diagonală.

**Reprezentarea soluției:**
Dacă fiecărei dame îi este prealocată o coloană distinctă (de la 0 la N-1), problema se reduce la a găsi pe ce rând (linie) se plasează fiecare damă, astfel încât nicio damă să nu fie pe același rând sau pe aceeași diagonală cu alta. O astfel de configurație poate fi reprezentată printr-o permutație de lungime N. De exemplu, pentru N=4, individul `[1, 3, 0, 2]` înseamnă:
*   Dama de pe coloana 0 este pe rândul 1.
*   Dama de pe coloana 1 este pe rândul 3.
*   Dama de pe coloana 2 este pe rândul 0.
*   Dama de pe coloana 3 este pe rândul 2.

Această reprezentare asigură automat că nu există două dame pe aceeași coloană și nici pe același rând.

**Funcția de fitness:**
Vom număra câte perechi de dame se atacă reciproc pe diagonală. Cu ajutorul algoritmului evolutiv, vom căuta configurația ce minimizează numărul total al conflictelor. Soluția optimă a problemei are 0 conflicte.

**Formularea ca problemă de optimizare:**
*   **Variabile de decizie:** O permutație a rândurilor pe tablă pentru fiecare damă (coloanele fiind fixe și distincte).
*   **Funcția obiectiv:** Minimizarea numărului total de conflicte pe diagonale.

In [ ]:
def checkConflicts(individual_permutation):
  """
  Calculează numărul de conflicte pe diagonale pentru o configurație dată.
  'individual_permutation' este o listă unde indexul reprezintă coloana
  iar valoarea reprezintă rândul damei.
  """
  n = len(individual_permutation)
  if n == 0:
      return (float('inf'),) # Caz special pentru individ gol

  # Vom folosi două array-uri pentru a număra damele pe fiecare diagonală.
  # Diagonalele stânga-sus spre dreapta-jos: suma coordonatelor (linie + coloană) este constantă.
  # Diagonalele dreapta-sus spre stânga-jos: diferența coordonatelor (linie - coloană) este constantă.
  # Pentru a evita indecși negativi, putem ajusta (linie - coloană + (n-1)).

  left_diagonal_counts = np.zeros(2 * n - 1, dtype=int) # Indecși de la 0 la 2n-2
  right_diagonal_counts = np.zeros(2 * n - 1, dtype=int) # Indecși de la 0 la 2n-2

  # Calculăm numărul de dame pe fiecare diagonală
  for col in range(n):
      row = individual_permutation[col]
      left_diagonal_counts[row + col] += 1
      right_diagonal_counts[row - col + (n - 1)] += 1 # Ajustare pentru a avea indecși pozitivi

  # Calculăm numărul de conflicte
  # Dacă pe o diagonală sunt k dame, atunci sunt k*(k-1)/2 perechi,
  # sau, mai simplu, (k-1) conflicte dacă numărăm fiecare damă implicată o dată după prima.
  numConflicts = 0
  for i in range(2 * n - 1):
      if left_diagonal_counts[i] > 1:
            numConflicts += (left_diagonal_counts[i] * (left_diagonal_counts[i] - 1)) / 2
      if right_diagonal_counts[i] > 1:
            numConflicts += (right_diagonal_counts[i] * (right_diagonal_counts[i] - 1)) / 2

  return numConflicts, # DEAP așteaptă un tuplu ca rezultat pentru fitness


Definim o funcție pentru afișarea (pe tablă) a unei configurații codificate.

In [ ]:
def displayBoard(individual_permutation):
  n = len(individual_permutation)
  board = [['.' for _ in range(n)] for _ in range(n)]

  # individual_permutation[col] = row
  for col in range(n):
    row = individual_permutation[col]
    # Inversăm afișarea rândurilor pentru o vizualizare standard a tablei de șah
    # (rândul 0 jos, rândul n-1 sus)
    # Dar, convențional, în programare, rândul 0 e sus. Vom păstra convenția de programare.
    # Dacă individual_permutation[col] = row, atunci board[row][col] = 'Q'
    board[row][col] = 'Q'

  # Afișăm tabla cu rândul 0 sus
  for i in range(n):
    print(' '.join(board[i]))
  print("-" * (2*n -1)) # Separator

Testarea funcțiilor `checkConflicts` și `displayBoard`:

In [ ]:
print("Test 1: Configurație cu multe conflicte (taote damele pe diagonala principală)")
ind_diag_principala = list(range(8)) # Dame pe diagonala principală (0,0), (1,1), ...
# Permutația corespunzătoare: [0, 1, 2, 3, 4, 5, 6, 7]
# Dama din col 0 e pe rândul 0, dama din col 1 e pe rândul 1 etc.
displayBoard(ind_diag_principala)
print(f"Conflicte: {checkConflicts(ind_diag_principala)[0]}") # Ar trebui să dea multe conflicte

print("\nTest 2: O soluție cunoscută pentru 8 dame")
ind_solutie_8_dame = [3, 6, 2, 7, 1, 4, 0, 5] # O soluție validă
# Dama col 0 -> rând 3
# Dama col 1 -> rând 6
# ...
# Dama col 7 -> rând 5
displayBoard(ind_solutie_8_dame)
print(f"Conflicte: {checkConflicts(ind_solutie_8_dame)[0]}") # Ar trebui să fie (0,)

print("\nTest 3: Configurații aleatoare")
for i in range(2): # Testăm cu 2 configurații aleatoare
  ind_aleator = list(np.random.permutation(8))
  displayBoard(ind_aleator)
  print(f"Conflicte: {checkConflicts(ind_aleator)[0]}")

Test 1: Configurație fără conflicte (teoretic, pentru diagonale principale)
Q . . . . . . .
. Q . . . . . .
. . Q . . . . .
. . . Q . . . .
. . . . Q . . .
. . . . . Q . .
. . . . . . Q .
. . . . . . . Q
---------------
Conflicte: 28.0

Test 2: O soluție cunoscută pentru 8 dame
. . . . . . Q .
. . . . Q . . .
. . Q . . . . .
Q . . . . . . .
. . . . . Q . .
. . . . . . . Q
. Q . . . . . .
. . . Q . . . .
---------------
Conflicte: 0

Test 3: Configurații aleatoare
. . . . . . . Q
. Q . . . . . .
Q . . . . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . Q . .
. . . Q . . . .
. . . . Q . . .
---------------
Conflicte: 5.0
Q . . . . . . .
. . . . . . . Q
. Q . . . . . .
. . . Q . . . .
. . Q . . . . .
. . . . Q . . .
. . . . . Q . .
. . . . . . Q .
---------------
Conflicte: 8.0


Definim lungimea "cromozomului" (individului). În formularea folosită, aceasta este egală cu numărul de dame (N), deoarece fiecare element al permutației reprezintă rândul unei dame de pe o coloană predefinită.

In [ ]:
NB_QUEENS = 8 # Numărul de dame (și dimensiunea tablei NxN)
IND_SIZE = NB_QUEENS # Lungimea permutației

Definim componentele algoritmului evolutiv în DEAP:
*   **Fitness:** Dorim să minimizăm numărul conflictelor, deci `weights=(-1.0,)`.
*   **Individ:** Un individ este o listă (care va conține o permutație) și are asociat un obiect fitness de tip `FitnessMin`.
*   **Populație:** O listă de indivizi.
*   **Funcții genetice:**
    *   `permutation`: Generează o permutație aleatoare de `IND_SIZE` elemente (de la 0 la `IND_SIZE-1`).
    *   `individual`: Creează un individ prin inițializarea sa cu o permutație generată de `toolbox.permutation`.
    *   `population`: Creează o populație de indivizi.
    *   `evaluate`: Funcția de evaluare (fitness), `checkConflicts`.
    *   `mate`: Operatorul de încrucișare. Pentru permutări, `tools.cxPartialyMatched` (PMX) este o alegere bună.
    *   `mutate`: Operatorul de mutație. `tools.mutShuffleIndexes` amestecă aleatoriu un subset de gene (elemente din permutație). `indpb` este probabilitatea ca fiecare genă să fie inclusă în subsetul de amestecat.
    *   `select`: Strategia de selecție. `tools.selTournament` cu `tournsize=4` este o alegere robustă.

In [ ]:
# Dorim să minimizăm numărul conflictelor
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# Reprezentare - un șir de numere (ce formează o permutație)
# Un individ este o listă, cu un atribut de fitness
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Generator de atribute: o permutație de la 0 la IND_SIZE-1
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)

# Inițializator de indivizi și populație
# tools.initIterate ia un container (creator.Individual) și o funcție (toolbox.indices)
# pentru a umple containerul.
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Înregistrarea funcției de evaluare
toolbox.register("evaluate", checkConflicts)

# Înregistrarea operatorilor genetici specifici pentru permutări
toolbox.register("mate", tools.cxPartialyMatched) # Partially Matched Crossover
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=2.0/IND_SIZE) # Shuffle Indexes Mutation

# Înregistrarea strategiei de selecție
toolbox.register("select", tools.selTournament, tournsize=4) # Turnir de mărime 4


Cu ajutorul modulului `Statistics` putem genera (și afișa) statistici despre procesul de optimizare.
`HallOfFame` reține cele mai bune soluții găsite de-a lungul generațiilor.

In [ ]:
stats = tools.Statistics(lambda ind: ind.fitness.values[0]) # Extragem prima (și singura) valoare a fitness-ului
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Reținem cele mai bune 5 soluții (cele cu fitness minim)
hof = tools.HallOfFame(5)

# Creăm o populație inițială
pop_deap = toolbox.population(n=100) # Populație de 100 de indivizi

# Rulăm algoritmul evolutiv
# eaSimple este un algoritm generational standard
# cxpb: probabilitatea de încrucișare
# mutpb: probabilitatea de mutație
# ngen: numărul de generații
pop_deap, logbook_deap = algorithms.eaSimple(pop_deap, toolbox,
                                  cxpb=0.7, mutpb=0.2, # Probabilități mai mari pentru operatori
                                  ngen=100, stats=stats,
                                  halloffame=hof, verbose=True)


gen	nevals	avg 	std    	min	max
0  	100   	4.76	1.78393	1  	11 
1  	69    	3.99	2.18858	1  	16 
2  	79    	3.63	2.30935	1  	14 
3  	79    	3.18	1.73424	1  	7  
4  	80    	3.42	2.1456 	1  	11 
5  	74    	2.69	1.70115	1  	6  
6  	73    	2.72	2.0203 	1  	9  
7  	83    	2.11	1.67269	1  	8  
8  	74    	1.91	1.69172	0  	8  
9  	71    	1.46	1.19516	0  	7  
10 	78    	1.59	1.46352	0  	6  
11 	73    	1.39	1.53555	0  	8  
12 	82    	1.15	1.59609	0  	8  
13 	76    	1.16	1.95305	0  	8  
14 	86    	0.75	1.52561	0  	7  
15 	72    	0.61	1.40638	0  	6  
16 	78    	0.59	1.60683	0  	10 
17 	80    	0.38	1.41972	0  	11 
18 	75    	0.59	1.37182	0  	6  
19 	86    	0.62	1.42674	0  	8  
20 	75    	0.6 	1.47648	0  	7  
21 	75    	0.72	1.63144	0  	7  
22 	68    	0.77	1.88072	0  	10 
23 	82    	0.94	2.01405	0  	8  
24 	69    	0.57	1.52483	0  	7  
25 	73    	1.02	1.87072	0  	8  
26 	84    	0.71	1.5511 	0  	9  
27 	62    	0.68	1.68452	0  	8  
28 	78    	0.67	1.57515	0  	7  
29 	79    	0.73	1.66045	0  	7  
30 	70  

In [ ]:
# Afișarea celor mai bune soluții găsite de algoritmul DEAP:

print("\nCele mai bune soluții găsite de DEAP:")
for i, sol_deap in enumerate(hof):
    print(f"\nSoluția DEAP #{i+1} (Fitness: {sol_deap.fitness.values[0]} conflicte):")
    displayBoard(sol_deap)


Cele mai bune soluții găsite de DEAP:

Soluția DEAP #1 (Fitness: 0.0 conflicte):
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
. Q . . . . . .
---------------

Soluția DEAP #2 (Fitness: 0.0 conflicte):
. . . . . . . Q
. Q . . . . . .
. . . Q . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
. . Q . . . . .
. . . . . Q . .
---------------

Soluția DEAP #3 (Fitness: 0.0 conflicte):
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. Q . . . . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
---------------

Soluția DEAP #4 (Fitness: 0.0 conflicte):
. . . Q . . . .
. . . . . . . Q
Q . . . . . . .
. . . . Q . . .
. . . . . . Q .
. Q . . . . . .
. . . . . Q . .
. . Q . . . . .
---------------

Soluția DEAP #5 (Fitness: 0.0 conflicte):
. . . Q . . . .
. Q . . . . . .
. . . . . . . Q
. . . . Q . . .
. . . . . . Q .
Q . . . . . . .
. . Q . . . . .
. . . . . Q . .
---------------


## Partea II: Reprezentarea cu chei aleatoare (Random Keys) și BRKGA

O alternativă la reprezentarea directă prin permutări este utilizarea **cheilor aleatoare**. Această tehnică oferă o modalitate flexibilă de a codifica soluții permutative, permițând utilizarea operatorilor genetici standard (cum ar fi crossover-ul uniform sau pe un singur punct) fără a genera soluții invalide (adică, rezultatul decodificat este întotdeauna o permutație validă).

### Ce sunt cheile aleatoare?

Un individ în reprezentarea cu chei aleatoare este un vector de numere reale, de obicei din intervalul `[0, 1)`. Lungimea vectorului este egală cu numărul de elemente care trebuie ordonate (de exemplu, `N` pentru problema celor `N` dame).

**Decodificarea cheilor aleatoare într-o permutație:**
Pentru a obține o permutație dintr-un vector de chei aleatoare, sortăm valorile cheilor și folosim indicii originali ai acestor valori sortate pentru a forma permutația.

**Exemplu de decodificare:**
Să presupunem că avem `N=4` elemente de ordonat și următorul vector de chei aleatoare:
`keys = [0.72, 0.15, 0.91, 0.34]`

1.  Asociem fiecare cheie cu indexul său original (0-indexat):
    *   `0.72` (elementul/coloana originală cu index 0)
    *   `0.15` (elementul/coloana originală cu index 1)
    *   `0.91` (elementul/coloana originală cu index 2)
    *   `0.34` (elementul/coloana originală cu index 3)

2.  Sortăm aceste perechi `(cheie, index_original)` în funcție de valoarea cheii, în ordine crescătoare:
    *   `(0.15, index 1)`
    *   `(0.34, index 3)`
    *   `(0.72, index 0)`
    *   `(0.91, index 2)`

3.  Permutația rezultată este secvența indicilor originali din perechile sortate:
    `permutation = [1, 3, 0, 2]`

Aceasta înseamnă că elementul original cu indexul 1 este primul în noua ordine, elementul original cu indexul 3 este al doilea, ș.a.m.d.
Pentru problema damelor, dacă `N=4`, permutația `[1, 3, 0, 2]` se interpretează ca:
*   Pe coloana 0 (primul element al permutației, care este `1`) se află dama de pe rândul `1`.
*   Pe coloana 1 (al doilea element al permutației, care este `3`) se află dama de pe rândul `3`.
*   Pe coloana 2 (al treilea element al permutației, care este `0`) se află dama de pe rândul `0`.
*   Pe coloana 3 (al patrulea element al permutației, care este `2`) se află dama de pe rândul `2`.
Mai precis, dacă permutația este `p = [p_0, p_1, ..., p_{N-1}]`, atunci dama de pe coloana `i` este pe rândul `p_i`. Interpretarea corectă pentru `[1,3,0,2]` este: dama de pe coloana 0 este pe rândul 1, dama de pe coloana 1 este pe rândul 3, dama de pe coloana 2 este pe rândul 0, iar dama de pe coloana 3 este pe rândul 2. Decodificarea cheilor aleatoare produce **direct** permutația `[rand, coloana]` dorită.

Avantajul principal este că operatorii genetici standard (ex: crossover uniform) aplicați pe vectorii de chei aleatoare vor produce întotdeauna vectori de chei valide, care la rândul lor vor decodifica întotdeauna permutări valide. Nu mai este nevoie de operatori de încrucișare și mutație specializați pentru permutări.

### Biased Random-Key Genetic Algorithm (BRKGA)

BRKGA este o metaheuristică evolutivă propusă de Gonçalves și Resende (2011) care utilizează reprezentarea cu chei aleatoare. Termenul **"biased" (părtinitor)** se referă la modul în care soluțiile noi sunt generate, cu o preferință (o "părtinire") pentru materialul genetic provenit de la soluțiile de elită (cele mai bune soluții găsite până la un moment dat).

**Componentele principale ale BRKGA:**

1.  **Populația:**
    *   Este împărțită în două (sau mai multe) grupuri. O structură tipică include:
        *   **Setul de elită (`n_elites`):** Conține un număr fix de cele mai bune soluții găsite până în prezent. Acestea sunt transmise nemodificate generației următoare.
        *   **Setul non-elită:** Restul populației, care nu sunt elite.
2.  **Generarea noii populații (pentru generația următoare):**
    *   **Elitism:** Indivizii din setul de elită sunt copiați direct în populația următoare.
    *   **Încrucișare (Crossover):** Un număr de urmași (`n_offsprings`) sunt generați. Pentru fiecare urmaș:
        *   Se selectează un părinte aleatoriu din setul de elită.
        *   Se selectează un părinte aleatoriu din setul non-elită (sau din întreaga populație, excluzând elitele).
        *   Se aplică un operator de încrucișare. Pentru BRKGA, se folosește frecvent **încrucișarea uniformă parametrizată (biased uniform crossover)**:
            *   Pentru fiecare genă (cheie aleatoare) din urmaș: cu o probabilitate `p_bias` (de ex. 0.7), gena este moștenită de la părintele de elită; altfel (cu probabilitatea `1 - p_bias`), este moștenită de la celălalt părinte. Această probabilitate `p_bias` introduce "părtinirea" către părintele de elită.
    *   **Mutanți (`n_mutants`):** Un număr de indivizi noi sunt generați complet aleatoriu (la fel ca în populația inițială). Aceștia ajută la menținerea diversității și la explorarea unor noi regiuni ale spațiului soluțiilor.

Mărimea totală a populației în BRKGA este de obicei `n_elites + n_offsprings + n_mutants`.

**Fluxul general al BRKGA:**
1.  Inițializează o populație de `P` indivizi (vectori de chei aleatoare).
2.  Pentru fiecare generație:
    a.  Decodifică fiecare individ (vector de chei) într-o soluție a problemei (ex: o permutație).
    b.  Evaluează funcția fitness pentru fiecare soluție decodificată.
    c.  Sortează populația pe baza valorilor fitness.
    d.  Identifică primii `n_elites` indivizi ca fiind setul de elită.
    e.  Construiește populația pentru generația următoare:
        i.  Copiază cei `n_elites` indivizi de elită.
        ii. Generează `n_offsprings` urmași prin încrucișare părtinitoare între un părinte de elită și un părinte non-elită.
        iii.Generează `n_mutants` indivizi noi complet aleatoriu.
3.  Repetă pasul 2 până la atingerea criteriului de oprire (ex: număr de generații, convergență).

BRKGA s-a dovedit eficient pentru o gamă largă de probleme de optimizare combinatorică.

### Demonstrație: Rezolvarea problemei celor N dame cu BRKGA folosind `pymoo`

Vom folosi biblioteca `pymoo` pentru a implementa BRKGA pentru problema damelor. Mai întâi, instalăm `pymoo`:


In [ ]:
!pip install pymoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.2 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=710e037a8143c5b69e80c0fe5bbc0a45679c0cf7fe40030f7010456a29db20b4
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


Importăm componentele necesare:

In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from pymoo.optimize import minimize
from pymoo.termination import get_termination # Pentru definirea criteriului de oprire
# Funcția displayBoard este definită anterior în secțiunea DEAP. O vom reutiliza.
# Funcția checkConflicts o vom adapta puțin pentru pymoo.

Vom reutiliza funcția `checkConflicts` definită anterior, dar o vom adapta ușor pentru `pymoo` (care așteaptă o singură valoare numerică pentru funcția obiectiv în problemele mono-obiectiv, nu un tuplu). De asemenea, avem nevoie de o funcție pentru a decodifica un vector de chei aleatoare într-o permutație.

In [ ]:
def decode_random_keys_to_permutation(random_keys_vector):
    """
    Decodifică un vector de chei aleatoare într-o permutație.
    Returnează o listă de indici care reprezintă permutația.
    """
    n_elements = len(random_keys_vector)
    # Creăm perechi (cheie, index_original)
    indexed_keys = []
    for i in range(n_elements):
        indexed_keys.append((random_keys_vector[i], i))

    # Sortăm perechile după valoarea cheii (primul element al tuplului)
    indexed_keys.sort(key=lambda x: x[0])

    # Permutația este formată din indicii originali, în noua ordine dată de sortare
    permutation = []
    for i in range(n_elements):
        permutation.append(indexed_keys[i][1])

    return permutation

# Adaptăm funcția checkConflicts pentru a returna o singură valoare, așa cum așteaptă pymoo.
# Și o redenumim pentru claritate.
def checkConflicts_for_pymoo(individual_permutation):
    return checkConflicts(individual_permutation)[0] # Pymoo așteaptă o valoare numerică, nu un tuplu


Acum definim problema pentru `pymoo`. O problemă în `pymoo` este o clasă care moștenește din `pymoo.core.problem.Problem`.
Trebuie să specificăm:
*   `n_var`: numărul de variabile de decizie (lungimea vectorului de chei aleatoare, adică `NB_QUEENS`).
*   `n_obj`: numărul de funcții obiectiv (1 în cazul nostru, minimizarea conflictelor).
*   `n_constr`: numărul de constrângeri (0 în cazul nostru, deoarece reprezentarea cu chei aleatoare garantează permutări valide prin decodificare).
*   `xl`, `xu`: limitele inferioare și superioare pentru variabilele de decizie (0.0 și 1.0 pentru cheile aleatoare).
*   `_evaluate`: metoda care calculează funcția obiectiv. Aceasta primește un batch de soluții `x` (unde fiecare rând este un individ/vector de chei aleatoare) și trebuie să returneze un dicționar `out` unde `out["F"]` conține valorile funcției obiectiv pentru fiecare individ.


In [ ]:
NB_QUEENS_BRKGA = 8 # Putem testa cu valori mai mari ulterior

class NQueensProblemBRKGA(Problem):
    def __init__(self, n_queens):
        super().__init__(n_var=n_queens,   # Numărul de variabile (lungimea vectorului de chei)
                         n_obj=1,          # O singură funcție obiectiv (minimizarea conflictelor)
                         n_constr=0,       # Fără constrângeri explicite aici
                         xl=0.0,           # Limita inferioară pentru cheile aleatoare
                         xu=1.0)           # Limita superioară pentru cheile aleatoare
        self.n_queens = n_queens

    def _evaluate(self, x_batch_keys, out, *args, **kwargs):
        # x_batch_keys este o matrice NumPy unde fiecare rând este un vector de chei aleatoare
        # (un individ din populație).

        objectives_list = []
        for individual_keys in x_batch_keys:
            # 1. Decodificăm vectorul de chei aleatoare într-o permutație
            permutation = decode_random_keys_to_permutation(individual_keys)

            # 2. Calculăm numărul de conflicte pentru permutația obținută
            conflicts = checkConflicts_for_pymoo(permutation)
            objectives_list.append(conflicts)

        # Rezultatele trebuie returnate într-un format specific pentru pymoo:
        # un array NumPy cu o coloană (pentru probleme mono-obiectiv).
        out["F"] = np.array(objectives_list).reshape(-1, 1)

# Inițializăm problema pentru pymoo
problem_brkga = NQueensProblemBRKGA(n_queens=NB_QUEENS_BRKGA)

Definim algoritmul BRKGA și parametrii săi.
Parametrii comuni pentru BRKGA în `pymoo`:
*   `n_elites`: numărul de indivizi de elită.
*   `n_offsprings`: numărul de urmași generați prin încrucișare.
*   `n_mutants`: numărul de mutanți (indivizi generați aleatoriu).
*   `bias`: probabilitatea `p_e` (sau `rho_e` în unele notații) de a alege gena de la părintele de elită în timpul încrucișării (de obicei în jur de 0.7).
*   `eliminate_duplicates`: o opțiune utilă pentru a menține diversitatea (poate fi costisitoare computațional pentru populații mari).

Mărimea totală a populației va fi `n_elites + n_offsprings + n_mutants`.

In [ ]:
# Setăm parametrii pentru BRKGA
pop_size_brkga = 100
n_elites_brkga = int(0.2 * pop_size_brkga)    # 20% elite
n_mutants_brkga = int(0.15 * pop_size_brkga)  # 15% mutanți
n_offsprings_brkga = pop_size_brkga - n_elites_brkga - n_mutants_brkga # Restul sunt urmași

algorithm_brkga = BRKGA(
    n_elites=n_elites_brkga,
    n_offsprings=n_offsprings_brkga,
    n_mutants=n_mutants_brkga,
    bias=0.7,  # Probabilitatea de a moșteni de la părintele de elită
    eliminate_duplicates=True # Poate ajuta la diversitate
)

# Definim criteriul de oprire (de ex., un număr de generații)
termination_brkga = get_termination("n_gen", 100) # Oprim după 100 de generații

# Rulăm optimizarea cu BRKGA
results_brkga = minimize(problem_brkga,
                         algorithm_brkga,
                         termination_brkga,
                         seed=1, # Pentru reproductibilitatea rezultatelor
                         verbose=True) # Afișează progresul

n_gen  |  n_eval  |     f_avg     |     f_min    
     1 |      100 |  6.0000000000 |  1.0000000000
     2 |      180 |  6.6666666667 |  1.0000000000
     3 |      260 |  7.3846153846 |  1.0000000000
     4 |      340 |  7.7142857143 |  1.0000000000
     5 |      420 |  7.7142857143 |  1.0000000000
     6 |      500 |  7.7142857143 |  1.0000000000
     7 |      580 |  8.2000000000 |  1.0000000000
     8 |      660 |  8.2000000000 |  1.0000000000
     9 |      740 |  8.2000000000 |  1.0000000000
    10 |      820 |  8.8125000000 |  1.0000000000
    11 |      900 |  8.2941176471 |  0.000000E+00
    12 |      980 |  8.5555555556 |  0.000000E+00
    13 |     1060 |  8.5555555556 |  0.000000E+00
    14 |     1140 |  8.5555555556 |  0.000000E+00
    15 |     1220 |  8.5555555556 |  0.000000E+00
    16 |     1300 |  8.5555555556 |  0.000000E+00
    17 |     1380 |  8.5555555556 |  0.000000E+00
    18 |     1460 |  8.5555555556 |  0.000000E+00
    19 |     1540 |  8.5555555556 |  0.000000E+00


Afișăm cea mai bună soluție găsită de BRKGA:

In [ ]:
print("\nRezultate BRKGA:")
if results_brkga.X is not None:
    # results_brkga.X este vectorul de chei aleatoare pentru cea mai bună soluție
    best_random_keys = results_brkga.X
    # results_brkga.F este valoarea funcției obiectiv pentru cea mai bună soluție
    best_fitness_value = results_brkga.F[0]

    # Decodificăm cheile pentru a obține permutația
    best_permutation_brkga = decode_random_keys_to_permutation(best_random_keys)

    print(f"Cele mai bune chei aleatoare găsite: {np.round(best_random_keys, 3)}")
    print(f"Permutația decodificată: {best_permutation_brkga}")
    print(f"Fitness (conflicte): {best_fitness_value}")

    if best_fitness_value == 0:
        print("Soluție optimă găsită!")
    else:
        print("Soluție sub-optimă găsită.")

    print("\nConfigurația pe tabla de șah:")
    displayBoard(best_permutation_brkga) # Reutilizăm funcția de afișare
else:
    print("BRKGA nu a returnat o soluție (posibil eroare sau nicio soluție fezabilă în contextul dat).")


Rezultate BRKGA:
Cele mai bune chei aleatoare găsite: [0.124 0.279 0.586 0.97  0.17  0.019 0.801 0.421]
Permutația decodificată: [5, 0, 4, 1, 7, 2, 6, 3]
Fitness (conflicte): 0.0
Soluție optimă găsită!

Configurația pe tabla de șah:
. Q . . . . . .
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
---------------


Comparativ cu reprezentarea directă prin permutări folosită în DEAP, BRKGA cu chei aleatoare oferă avantajul că operatorii de încrucișare standard (cum ar fi cel uniform parametrizat, specific BRKGA) pot fi aplicați direct pe vectorii de chei aleatoare, generând mereu urmași valizi (care decodifică la permutări valide). Aceasta simplifică designul algoritmului, deoarece nu este nevoie de operatori de încrucișare specializați pentru permutări (precum PMX, CX, OX etc.). Dezavantajul ar putea fi un efort computațional suplimentar necesar pentru decodificarea cheilor în permutări la fiecare evaluare a funcției fitness. Alegerea reprezentării și a algoritmului depinde adesea de specificul problemei și de ușurința implementării.

### Alte probleme de optimizare cu reprezentare prin permutări (idei și pentru proiect)

Există numeroase [probleme interesante](https://link.springer.com/book/10.1007/978-3-540-92151-6) și cu aplicații practice importante în informatică și optimizare combinatorică în care soluțiile optime sunt [permutări](http://web.fdi.ucm.es/posgrado/conferencias/JosuCeberio2018-slides.pdf). Iată doar câteva exemple:

1.  **Problema Comis-Voiajorului (Traveling Salesperson Problem - TSP):** Găsirea celui mai scurt tur care vizitează un set de orașe exact o dată și se întoarce la orașul de start. Soluția este o permutație a orașelor.
2.  **Job Scheduling Problem** - problema de programare a joburilor: Avem un set de joburi ce trebuie procesate pe un set de mașini. Scopul este de a găsi o secvență optimă (permutare) a joburilor care minimizează timpul total de finalizare, makespan-ul sau alte criterii. Variantele includ:
    *   **Single-Machine Scheduling**: Programarea pe o singură mașină.
    *   **Flow Shop Scheduling**: Joburile trec prin mai multe mașini în aceeași ordine.
    *   **Job Shop Scheduling**: Joburile trec prin mașini într-o ordine specifică, unică pentru fiecare job.
3.  **Vehicle Routing Problem (VRP)** - Problema de rutare a vehiculelor: Generalizare a TSP, găsirea rutelor optime pentru o flotă de vehicule pentru a deservi un set de clienți.
4.  **Quadratic Assignment Problem (QAP)** - Problema de alocare pătratică: Alocarea unui set de facilități într-un set de locații pentru a minimiza un cost total bazat pe distanțe și fluxuri între facilități.
5.  **Optimal Assembly Line Balancing** - Echilibrarea optimă a liniei de asamblare: Atribuirea sarcinilor la stații de lucru pentru a minimiza timpii morți.
6.  **Linear Arrangement Problem** - Problema de aranjare liniară: Ordonarea vârfurilor unui graf pentru a minimiza suma lungimilor muchiilor.

# Exerciții

1.  **Înțelegerea operatorilor DEAP:**
    *   Cum funcționează operatorul de încrucișare `tools.cxPartialyMatched` (PMX) folosit în exemplul DEAP? Descrieți mecanismul său pas cu pas pe un exemplu mic (ex: doi părinți cu 5 gene).
    *   Cum funcționează operatorul de mutație `tools.mutShuffleIndexes`? Ce efect are parametrul `indpb`?

2.  **Alți operatori pentru permutări:** Ce alți operatori de încrucișare (ex: Order Crossover - OX, Cycle Crossover - CX) și de mutație (ex: Swap Mutation, Insert Mutation, Scramble Mutation) ar fi compatibili cu reprezentarea de tip permutație în DEAP? Descrieți pe scurt cum funcționează unul dintre acești operatori de încrucișare și unul de mutație, la alegere.

3.  **Scalabilitatea soluției DEAP pentru N-dame:**
    *   Modificați valoarea `NB_QUEENS` în secțiunea DEAP. Până la ce număr de dame (`NB_QUEENS`) puteți obține soluții optime (fitness de 0 conflicte) în mod stabil (adică metoda găsește măcar o soluție optimă la majoritatea rulărilor, să zicem în 2 din 3 rulări), menținând parametrii actuali ai algoritmului (populație de 100, 100 generații)?
    *   Documentați încercările și rezultatele.

4.  **Scalabilitatea soluției BRKGA pentru N-dame:**
    *   Similar exercițiului anterior, modificați `NB_QUEENS_BRKGA` în secțiunea `pymoo`. Până la ce număr de dame puteți obține soluții optime în mod stabil cu BRKGA, menținând parametrii actuali (populație de 100, 100 generații, `bias=0.7`)?
    *   Pentru o versiune mai mare a problemei (ex: N=20 sau N=32 dame), experimentați cu mărimea populației (`pop_size_brkga`) și numărul de generații (`termination_brkga`) în BRKGA. Puteți găsi soluții optime? Ce observați legat de timpul de rulare?

5.  **Operare manuală cu chei aleatoare:**
    *   **Decodificare:** Fie vectorul de chei aleatoare `K = [0.4, 0.8, 0.1, 0.5, 0.2]` pentru `N=5`. Decodificați manual acest vector pentru a obține permutația corespunzătoare. Arătați pașii.
    *   **Încrucișare:** Fie doi părinți reprezentați prin chei aleatoare pentru `N=4`:
        *   `P1_keys = [0.6, 0.2, 0.9, 0.4]`
        *   `P2_keys = [0.3, 0.7, 0.1, 0.8]`
        Presupunând o încrucișare uniformă parametrizată cu `p_bias = 0.6` (probabilitatea de a alege gena de la `P1_keys` - considerat părintele de elită) și următorul șir de numere aleatoare `R = [0.5, 0.7, 0.2, 0.9]` (un număr aleator pentru fiecare genă, comparat cu `p_bias`), generați manual vectorul de chei aleatoare al urmașului.

6.  **Comparație cu Backtracking:**
    *   Cum se compară timpul de rulare al algoritmilor evolutivi (DEAP/BRKGA) pentru problema N-damelor față de o căutare exhaustivă de tip [backtracking](https://www.geeksforgeeks.org/n-queen-problem-backtracking-3/)?
    *   Pentru N mic (ex: N=8, N=10), care metodă credeți că ar fi mai rapidă? Dar pentru N mare (ex: N=30, N=50)? Argumentați.
    *   Care sunt avantajele și dezavantajele fiecărei abordări (evolutivă vs. backtracking) pentru această problemă?

7.  **Modificarea reprezentării în DEAP:**
    *   Modificați rezolvarea problemei N-damelor în DEAP, trecând de la o reprezentare de tip permutație la una de tip listă de `N` numere întregi, unde fiecare număr `i` din listă (de la indexul `j`) reprezintă rândul damei de pe coloana `j` (similar cu interpretarea permutației, dar fără constrângerea ca lista să fie o permutație). Valorile din listă pot fi între `0` și `N-1` și se pot repeta.
    *   Adaptați funcția de fitness pentru a penaliza și damele aflate pe același rând sau coloană.
    *   Folosiți operatori genetici standard pentru liste de întregi (ex: `tools.cxTwoPoint`, `tools.mutUniformInt`).
    *   Cum afectează această reprezentare "mai liberă" (dar care poate genera soluții cu conflicte pe rânduri, coloane) procesul de căutare și convergența metodei? Comparați cu abordarea bazată pe permutări.

8.  **Rezolvarea unei alte probleme:**
    *   Alegeți o problemă din lista "Alte probleme de optimizare cu reprezentare prin permutări" (sau o altă problemă adecvată, la alegere).
    *   Implementați o soluție folosind DEAP cu reprezentare prin permutări.
    *   Implementați o soluție folosind `pymoo` cu reprezentare prin chei aleatoare și BRKGA.
    *   Comparați cele două abordări din punct de vedere al calității soluțiilor, timpului de convergență și ușurinței implementării pentru problema aleasă. Documentați-vă și prezentați problema, modelarea matematică, funcția de fitness și rezultatele empirice obținute.


Ex5.

valoare  indice original

0.1   |  2

0.2   |  4

0.4   |  0

0.5   |  3

0.8   |  1


idx  →  0   1   2   3   4

rang →  3   5   1   4   2

[0.6, 0.7, 0.9, 0.8]

Ex6.

Pentru N-queens mici (sub 10), backtracking e foarte rapid si gaseste toate solutiile, fara sa ai de ajustat niciun parametru. Insa cand N creste, timpul explodeaza si devine imposibil de folosit. Algoritmii evolutivi, in schimb, lucreaza in timp rezonabil chiar si pentru N mai mari (30, 50), gasind o solutie fara conflicte de fiecare data, desi nu garanteaza ca le gasesc pe toate si trebuie sa alegi valori pentru populatie, mutatie si incrucisare. Astfel, backtrackingul e ideal cand vrei exactitate si N mic, iar evolutionarii sunt mai potriviti cand ai nevoie de o rezolvare rapida la N mari.